<a href="https://colab.research.google.com/github/Apple03244/Colaboratory/blob/main/Kaggle_TiTanic_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import google.colab as colab
f=colab.files.upload()

Saving kaggle.json to kaggle (1).json


In [29]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [30]:
!kaggle competitions download -c titanic

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [31]:
# zip 파일로 다운받아졌으므로
! unzip /content/titanic.zip

Archive:  /content/titanic.zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gender_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [32]:
import pandas as pd
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
sub_mission=pd.read_csv('/content/gender_submission.csv')

In [33]:
import sklearn.preprocessing as skpre
import sklearn.model_selection as skmod
import sklearn.impute as skimp
import sklearn.compose as skcom
import sklearn.pipeline as skpip
import sklearn.multioutput as skmult

In [34]:
# 승객번호는 나에게 필요없다
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


## preprocessing
- 1. Nan values

In [36]:
target_col=["Survived"]
numeric_cols=["Age","SibSp","Parch","Fare"]
cate_cols=["Pclass","Sex","Embarked"]

In [37]:
import numpy as np

1. use KNN imputer to predict Nan values

We need to consider what is best method for Nan value

So, do cross validation test for each model which predict these features

also, consider how many neighbors are needed for these method

In [50]:
import sklearn.neighbors as sknei
nei_num=5
knn_imputer=skimp.KNNImputer(n_neighbors=nei_num,weights='distance')

validation test

In [53]:
# data for cross validation
sub_train=train[numeric_cols].dropna(axis=0)

In [64]:
# need to make Nan value in sub_train


----

### Make pipeline

- 1. numerical_columns : KNN Impute -> MinMaxScaler

- 2. categorical_columns : most_frequent Impute -> Onehot Encoder

In [117]:
num_process=skpip.make_pipeline(skimp.KNNImputer(n_neighbors=5),
                                skpre.MinMaxScaler())
cate_process=skpip.make_pipeline(skimp.SimpleImputer(strategy="most_frequent")
          ,skpre.OneHotEncoder(drop="first",handle_unknown='ignore'))

import sklearn.base as skbas

class PassThrough(skbas.TransformerMixin,skbas.BaseEstimator):
  def fit(self, X, y=None):
          return self

  def transform(self, X):
      return X

In [118]:
total_process=skcom.make_column_transformer((num_process,numeric_cols),
                                            (cate_process,["Sex","Embarked"]),
                                            (PassThrough(),["Pclass","Survived"]),
                                            remainder="drop")

In [119]:
total_process.fit(X=train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('knnimputer', KNNImputer()),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['Age', 'SibSp', 'Parch', 'Fare']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked']),
                                ('passthrough', PassThrough(),
                                 ['Pclass', 'Survived'])])

In [123]:
# for getting feature names
fitted_cate_pipeline=total_process.named_transformers_['pipeline-2']
cate_cols_names=list(fitted_cate_pipeline.get_feature_names_out())

In [124]:
preprocessed_train=pd.DataFrame(total_process.transform(train),
                                columns=list(numeric_cols)+cate_cols_names+["Pclass","Survived"])

In [125]:
preprocessed_train

,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,Pclass,Survived
0,0.271174,0.125,0.000000,0.014151,1.0,0.0,1.0,3.0,0.0
1,0.472229,0.125,0.000000,0.139136,0.0,0.0,0.0,1.0,1.0
2,0.321438,0.000,0.000000,0.015469,0.0,0.0,1.0,3.0,1.0
3,0.434531,0.125,0.000000,0.103644,0.0,0.0,1.0,1.0,1.0
4,0.434531,0.000,0.000000,0.015713,1.0,0.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...
886,0.334004,0.000,0.000000,0.025374,1.0,0.0,1.0,2.0,0.0
887,0.233476,0.000,0.000000,0.058556,0.0,0.0,1.0,1.0,1.0
888,0.331490,0.125,0.333333,0.045771,0.0,0.0,1.0,3.0,0.0
889,0.321438,0.000,0.000000,0.058556,1.0,0.0,0.0,1.0,1.0
